# PyTerrier Notebook for Full-Rank Submissions

This notebook serves as a baseline full-rank submission for [TIRA](https://tira.io)/[TIREx](https://tira.io/tirex) that builds a PyTerrier index and subsequently creates a run with BM25.

### Step 1: Ensure Libraries are Imported

In [47]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !python -m pip install --upgrade pip
    !pip3 install  python-terrier tira==0.0.88 ir_datasets
    !pip3 install -q python-terrier
    !pip3 install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
else:
    print('We are in the TIRA sandbox.')

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

Ensure Pyterrier integration is loaded

In [48]:
ensure_pyterrier_is_loaded()

Due to execution in TIRA, I have patched ir_datasets to always return the single input dataset mounted to the sandbox.


### Step 2: Load data, create index

In [49]:
# dataset_name = 'validation-20231104-training'
jena = 'jena-topics-20231026-test'
leipzig = 'leipzig-topics-small-20240119-training'
ds_jena = pt.get_dataset('irds:ir-lab-jena-leipzig-wise-2023/'+jena)
ds_leipzig = pt.get_dataset('irds:ir-lab-jena-leipzig-wise-2023/'+leipzig)
topics_jena = ds_jena.get_topics(variant="title")
qrels_jena = ds_jena.get_qrels()
topics_leipzig = ds_leipzig.get_topics(variant="title")
qrels_leipzig = ds_leipzig.get_qrels()

In [50]:
from tira.rest_api_client import Client
tira_client = Client()

No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


In [51]:
bm25_bo1_pl2_max_j = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/mild-duck', jena) 
max_run_j = bm25_bo1_pl2_max_j.transform(topics_jena)

bm25_bo1_pl2_max_l = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/mild-duck', leipzig) 
max_run_l = bm25_bo1_pl2_max_l.transform(topics_leipzig)

In [52]:
bm25_bo1_pl2_mean_j = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/poky-claim', jena)
mean_run_j = bm25_bo1_pl2_mean_j.transform(topics_jena)

bm25_bo1_pl2_mean_l = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/poky-claim', leipzig)
mean_run_l = bm25_bo1_pl2_mean_l.transform(topics_leipzig)

In [53]:
bm25_bo1_pl2_kmax_j = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/edible-status', jena)
kmax_run_j = bm25_bo1_pl2_kmax_j.transform(topics_jena)

bm25_bo1_pl2_kmax_l = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/edible-status', leipzig)
kmax_run_l = bm25_bo1_pl2_kmax_l.transform(topics_leipzig)

### Experiments on Jena topics.

In [61]:
experiment_j = pt.Experiment(
        [
            bm25_bo1_pl2_max_j,
            bm25_bo1_pl2_mean_j,
            bm25_bo1_pl2_kmax_j,
        ],
        topics_jena,
        qrels_jena,
        eval_metrics=["ndcg_cut_5", "ndcg_cut_10", "ndcg"],
        names=[
            "max passage Jena",
            "mean passage Jena",
            "best kmax passage Jena",
        ],
        # perquery=True
    )

hypo1_j = pt.Experiment(
    [bm25_bo1_pl2_max_j, bm25_bo1_pl2_mean_j],
    topics_jena,
    qrels_jena,
    eval_metrics=["ndcg_cut_5", "ndcg_cut_10", "ndcg"],
    names=["max passage Jena", "mean passage Jena"],
    baseline = 1
)

hypo2_j = pt.Experiment(
    [bm25_bo1_pl2_kmax_j, bm25_bo1_pl2_max_j, bm25_bo1_pl2_mean_j],
    topics_jena,
    qrels_jena,
    eval_metrics=["ndcg_cut_5", "ndcg_cut_10", "ndcg"],
    names=["best kmax passage Jena", "max passage Jena", "mean passage Jena"],
    baseline = 0
)

experiment_l = pt.Experiment(
        [
            bm25_bo1_pl2_max_l,
            bm25_bo1_pl2_mean_l,
            bm25_bo1_pl2_kmax_l,
        ],
        topics_leipzig,
        qrels_leipzig,
        eval_metrics=["ndcg_cut_5", "ndcg_cut_10", "ndcg"],
        names=[
            "max passage Leipzig",
            "mean passage Leipzig",
            "best kmax passage Leipzig",
        ],
        # perquery=True
    )

hypo1_l = pt.Experiment(
    [bm25_bo1_pl2_max_l, bm25_bo1_pl2_mean_l],
    topics_leipzig,
    qrels_leipzig,
    eval_metrics=["ndcg_cut_5", "ndcg_cut_10", "ndcg"],
    names=["max passage Leipzig", "mean passage Leipzig"],
    baseline = 1
)

hypo2_l = pt.Experiment(
    [bm25_bo1_pl2_kmax_l, bm25_bo1_pl2_max_l, bm25_bo1_pl2_mean_l],
    topics_leipzig,
    qrels_leipzig,
    eval_metrics=["ndcg_cut_5", "ndcg_cut_10", "ndcg"],
    names=["best kmax passage Leipzig", "max passage Leipzig", "mean passage Leipzig"],
    baseline = 0
)

experiment_j

,name,ndcg_cut_5,ndcg_cut_10,ndcg
0,max passage Jena,0.268758,0.285880,0.373611
1,mean passage Jena,0.255592,0.279602,0.369422
2,best kmax passage Jena,0.283274,0.307513,0.385448


In [55]:
experiment_l

,name,ndcg_cut_5,ndcg_cut_10,ndcg
0,max passage Leipzig,0.581550,0.525421,0.573446
1,mean passage Leipzig,0.418526,0.423454,0.507939
2,best kmax passage Leipzig,0.539375,0.503897,0.560287


### Persist experiment and tests.

In [62]:
hypo1_j

,name,ndcg,ndcg_cut_5,ndcg_cut_10,ndcg +,ndcg -,ndcg p-value,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,max passage Jena,0.373611,0.268758,0.285880,15.0,19.0,0.819999,14.0,12.0,0.630029,12.0,19.0,0.790391
1,mean passage Jena,0.369422,0.255592,0.279602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
hypo1_l

,name,ndcg,ndcg_cut_5,ndcg_cut_10,ndcg +,ndcg -,ndcg p-value,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,max passage Leipzig,0.573446,0.581550,0.525421,26.0,7.0,0.004448,24.0,6.0,0.000489,26.0,6.0,0.001255
1,mean passage Leipzig,0.507939,0.418526,0.423454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
hypo2_j

,name,ndcg,ndcg_cut_5,ndcg_cut_10,ndcg +,ndcg -,ndcg p-value,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,best kmax passage Jena,0.385448,0.283274,0.307513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,max passage Jena,0.373611,0.268758,0.285880,12.0,20.0,0.147035,7.0,13.0,0.266367,11.0,16.0,0.052215
2,mean passage Jena,0.369422,0.255592,0.279602,14.0,19.0,0.321049,9.0,14.0,0.311403,15.0,13.0,0.177533


In [59]:
hypo2_l

,name,ndcg,ndcg_cut_5,ndcg_cut_10,ndcg +,ndcg -,ndcg p-value,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,best kmax passage Leipzig,0.560287,0.539375,0.503897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,max passage Leipzig,0.573446,0.581550,0.525421,18.0,13.0,0.328876,16.0,10.0,0.087884,16.0,14.0,0.252585
2,mean passage Leipzig,0.507939,0.418526,0.423454,8.0,26.0,0.002114,7.0,25.0,0.000748,8.0,25.0,0.001160


In [60]:
with open("results.txt", "wt") as file:
    file.write("On Jena topics:\n\n")
    file.write("Experiment:\n\n")
    experiment_string = experiment_j.to_string(header=True, index=False)
    file.write(experiment_string + "\n\n")
    # ...
    file.write("Results for Hypothesis 1:\n\n")
    file.write("Significance test:\n")
    hypo1_string = hypo1_j.to_string(header=True, index=False)
    file.write(hypo1_string + "\n\n")
    # ...
    file.write("Results for Hypothesis 2:\n\n")
    file.write("Significance test:\n")
    hypo2_string = hypo2_j.to_string(header=True, index=False)
    file.write(hypo2_string)
    # ...

    file.write("On Leipzig topics:\n\n")
    file.write("Experiment:\n\n")
    experiment_string = experiment_l.to_string(header=True, index=False)
    file.write(experiment_string + "\n\n")
    # ...
    file.write("Results for Hypothesis 1:\n\n")
    file.write("Significance test:\n")
    hypo1_string = hypo1_l.to_string(header=True, index=False)
    file.write(hypo1_string + "\n\n")
    # ...
    file.write("Results for Hypothesis 2:\n\n")
    file.write("Significance test:\n")
    hypo2_string = hypo2_l.to_string(header=True, index=False)
    file.write(hypo2_string)